# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


In [0]:
!pip install numpy==1.16.2
import numpy as np
print(np.__version__)

1.16.2


#### Usage:

In [0]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000
maxlen = 300 #number of word used from each review
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000, 300)
(25000,)
(25000, 300)
(25000,)


In [0]:
import pandas as pd
pd.Series(y_train).value_counts()

1    12500
0    12500
dtype: int64

In [0]:
print("Maximum value of a word index ")
print(max([max(sequence) for sequence in x_train]))
print("Maximum length num words of review in train ")
print(max([len(sequence) for sequence in x_train]))

Maximum value of a word index 
9999
Maximum length num words of review in train 
300


# Get the word index and then Create key value pair for word and word_id. (12.5 points)

In [0]:
INDEX_FROM=3   # word index offset
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["[PAD]"] = 0
#word_to_id[""] = 0
word_to_id["[🏃]"] = 1
word_to_id["[❓]"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}

def restore_original_text(imdb_x_array):
    return (' '.join(id_to_word[id] for id in imdb_x_array ))

In [0]:
id_to_word

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np

tf.set_random_seed(100)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                1088      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [0]:
history = model.fit(x_train,y_train,epochs=20,batch_size=512,validation_data=(x_test, y_test),verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 1s 29us/sample - loss: 0.6924 - acc: 0.5219 - val_loss: 0.6887 - val_acc: 0.6210
Epoch 2/20
25000/25000 [==============================] - 1s 21us/sample - loss: 0.6489 - acc: 0.7018 - val_loss: 0.5592 - val_acc: 0.8185
Epoch 3/20
25000/25000 [==============================] - 1s 22us/sample - loss: 0.4249 - acc: 0.8435 - val_loss: 0.3390 - val_acc: 0.8640
Epoch 4/20
25000/25000 [==============================] - 1s 22us/sample - loss: 0.2794 - acc: 0.8936 - val_loss: 0.3006 - val_acc: 0.8775
Epoch 5/20
25000/25000 [==============================] - 1s 22us/sample - loss: 0.2238 - acc: 0.9185 - val_loss: 0.2963 - val_acc: 0.8812
Epoch 6/20
25000/25000 [==============================] - 1s 22us/sample - loss: 0.1901 - acc: 0.9332 - val_loss: 0.2964 - val_acc: 0.8828
Epoch 7/20
25000/25000 [==============================] - 1s 21us/sample - loss: 0.1622 - acc: 0.9470 - val_loss: 0.

# Report the Accuracy of the model.

In [0]:
print('\nModel Performance: Log Loss and Accuracy on train data')
print(model.evaluate(x_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(x_test, y_test, batch_size = 20))


Model Performance: Log Loss and Accuracy on train data
25000/25000 [==============================] - 2s 76us/sample - loss: 0.0355 - acc: 0.9938
[0.03546932641845196, 0.99384]

Model Performance: Log Loss and Accuracy on validation data
25000/25000 [==============================] - 2s 76us/sample - loss: 0.6190 - acc: 0.8566
[0.6189705888716504, 0.85656]


In [0]:
import random
import pandas as pd

original_x_test = x_test
def restore_original_text(imdb_x_array):
    return (' '.join(id_to_word[id] for id in imdb_x_array ))
def imdb_class_to_str(imdb_class):
    if imdb_class == 0:
        return 'negative'
    return 'positive'
right = 0
mistake = 0
index_list = []
original_text_list = []
pred_prob_list = []
pred_class_list = []
y_test_list = []
fail_str_list = []

for i in range(100):
    index = random.randint(0, len(x_test))
    pred_prob = model.predict(x_test[index:(index+1)])[0][0] 
    pred_class = model.predict_classes(x_test[index:(index+1)])[0][0]
    
    '''
    print('pred_prod:', pred_prod)
    print('pred_class:', pred_class)
    print('y_test[index] :', y_test[index])
    '''
    fail_str = '' 
    
    if y_test[index] == pred_class:
        right += 1
    else:
        mistake += 1
        fail_str = 'Fail'
        
    original_text = restore_original_text(original_x_test[index])

    index_list.append(index)
    original_text_list.append(original_text)
    pred_prob_list.append(pred_prob)
    pred_class_list.append(imdb_class_to_str(pred_class))
    y_test_list.append(imdb_class_to_str(y_test[index]))
    fail_str_list.append(fail_str)

print("right : ", right)
print("mistake : ", mistake)
print("accuracy:", right/(right+mistake))

df = pd.DataFrame({'index': index_list, 
                   'x_test_original_text': original_text_list, 
                   'probability': pred_prob_list, 
                   'pred_class': pred_class_list,
                   'y_test': y_test_list,
                   'is_fail': fail_str_list
                  })

df[['index', 'x_test_original_text','probability','pred_class','y_test','is_fail']]

right :  79
mistake :  21
accuracy: 0.79


,index,x_test_original_text,probability,pred_class,y_test,is_fail
0,24721,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.981749,positive,negative,Fail
1,19106,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.995011,positive,negative,Fail
2,4878,with the others br br these folks have a [❓] [...,0.014484,negative,negative,
3,23569,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.999987,positive,positive,
4,24026,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.999987,positive,positive,
...,...,...,...,...,...,...
95,10894,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.003484,negative,negative,
96,4255,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.001289,negative,negative,
97,3060,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.000585,negative,negative,
98,2652,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0.998879,positive,positive,


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    def getLayerOutput(layer):
        get_Layer_Output = K.function([model.layers[0].input], [layer.output])
        return get_Layer_Output([x_test[0:1,]])[0]
    
    layer_output = []
    
    for layer in model.layers:
        layer_output.append(getLayerOutput(layer))

In [0]:
#Get the count of ouput. It should be equal to the number of layers
len(layer_output)

7

In [0]:
#Check all the outputs
layer_output

[array([[[ 0.02958138, -0.03169712,  0.01639029, ...,  0.01840125,
           0.01673038, -0.00414914],
         [ 0.02958138, -0.03169712,  0.01639029, ...,  0.01840125,
           0.01673038, -0.00414914],
         [ 0.02958138, -0.03169712,  0.01639029, ...,  0.01840125,
           0.01673038, -0.00414914],
         ...,
         [-0.02412609, -0.02690674, -0.02780943, ...,  0.04688034,
          -0.04966868, -0.01035297],
         [ 0.0172695 ,  0.03720633,  0.04049506, ...,  0.01201037,
           0.01892859, -0.0087445 ],
         [ 0.04296752,  0.02914286,  0.02458775, ..., -0.04049265,
          -0.0415541 , -0.04756479]]], dtype=float32),
 array([[ 0.02226276, -0.02361457,  0.01227525,  0.01205784, -0.03103019,
          0.02708768, -0.0201551 , -0.00823088,  0.02568788,  0.02446708,
         -0.00135381,  0.00020296, -0.02194796,  0.0150988 ,  0.01201752,
         -0.00423129]], dtype=float32),
 array([[0.00366598, 0.00291715, 0.02118429, 0.        , 0.01353196,
         0.01

In [0]:
#Get the specific layer output
layer_output[3]

array([[0.00366598, 0.00291715, 0.02118429, 0.        , 0.01353196,
        0.0167684 , 0.0097555 , 0.        , 0.        , 0.01148679,
        0.01258116, 0.        , 0.        , 0.        , 0.00023656,
        0.00308589, 0.00806987, 0.        , 0.01582277, 0.        ,
        0.00601472, 0.00916028, 0.00358524, 0.00741498, 0.01567596,
        0.        , 0.00980865, 0.        , 0.        , 0.00613568,
        0.00246035, 0.        , 0.00679383, 0.        , 0.        ,
        0.        , 0.00501485, 0.        , 0.00712595, 0.02079488,
        0.        , 0.01304207, 0.        , 0.01876357, 0.0098277 ,
        0.00481613, 0.0083033 , 0.        , 0.00395865, 0.01281297,
        0.        , 0.01482366, 0.01190546, 0.        , 0.00053326,
        0.0050906 , 0.        , 0.01198654, 0.00984045, 0.01865073,
        0.        , 0.        , 0.01072779, 0.00256948]], dtype=float32)